# Générons une prédiction valide pour le challenge

Imports nécessaires au notebook

In [78]:
import time
import re
import pandas as pd
import numpy as np
import nltk
import sklearn

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

Commencez par définir le nom de votre équipe termes alpha numériques sans espace

In [4]:
TEAM_NAME = "Wasabi"
assert re.match("^\w+$", TEAM_NAME) is not None, "Nom d'équipe invalide"

In [5]:
# Chargement des données d'entrainement
train = pd.read_csv("train.csv.gz")
train.head()

,idp,title,description,price,category
0,63610,4pairs silicone Oreillettes de remplacement du...,4pairs silicone Oreillettes de remplacement du...,4.90,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
1,580661,Dvd Rafaela Legouvello,Une aventure humaine hors du commun...,14.89,DVD - BLU-RAY > DVD > DVD DOCUMENTAIRE
2,90191,"Q2671a (H.309Ac) Toner Laser Hp Bleu (Cyan), ...","Q2671A (H.309AC) TONER LASER HP Bleu (Cyan),...",53.40,INFORMATIQUE > IMPRESSION - SCANNER > TONER - ...
3,1297725,Azalaïs ou La vie courtoise,Maryse Rouy,19.05,LIBRAIRIE > LITTERATURE > ROMANS HISTORIQUES
4,119129,Hamecon Coup Vmc Crystal 9408 Bronze (50 N°8),Hameçon coup Crystal 9408 bronzé VMC. Forme ...,7.59,SPORT > PECHE > HAMEÇON


In [109]:
# Chargement des données à prédire
test = pd.read_csv("test.csv.gz")
test.head()

KeyboardInterrupt: 

Combien y a t'il de catégories produits distinctes dans le dataset?

In [8]:
len(train.category.value_counts())

600

Ca fait beaucoup! Jusqu'ici vous n'aviez que 2 classes à prédire.

## Baseline : 
On va commencer par la prédiction la plus simple possible, une prédiction constante.

On prédit que toutes les catégories sont égales à la 1ère catégorie du dataset de train

In [9]:
category = train.iloc[0].category
print(f"On va toujours prédire: '{category}'")

On va toujours prédire: 'TV - VIDEO - SON > CASQUE - MICROPHONE - DICTAPHONE > CASQUE - ECOUTEURS'


sklearn a même un modèle tout fait pour se genre de cas d'usage : DummyClassifier

In [10]:
model = DummyClassifier(strategy='constant', constant=category)
model.fit(train["title"], train["category"])
y_pred = model.predict(test["title"])

et voici notre prédiction dans le bon format : 2 colonnes, l'identifiant produit suivi de sa catégorie

In [11]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})
prediction.head()

,idp,category
0,4873686,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
1,6320508,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
2,6351042,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
3,3853418,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
4,6373582,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...


## La fonction de publication de vos prédictions

In [12]:
# predictions est un dataframe qui contient au moins les colonnes "idp" et "category"
# Ca écrit simplement le fichier dans le répertoire d'évaluation
def publish_results(predictions):
    now = int(time.time())
    assert re.match("^\w+$", TEAM_NAME) is not None
    filename = f"/home/cisd-jacq/projet/evaluation/prediction-{TEAM_NAME}-{now}.csv.gz"
    predictions[["idp", "category"]].to_csv(filename, index=False, compression="gzip")
    return filename.split("/")[-1]

Une fois que vous avez spécifié un nom d'équipe vous pouvez soumettre votre 1ère prédiction en exécutant la cellule suivante.

In [99]:
publish_results(prediction)

'prediction-Wasabi-1578308823.csv.gz'

## La fonction d'évaluation de l'erreur

Vous pouvez utiliser cette fonction pour estimer sur un sous ensemble du dataset train quelle est la précision de votre modèle.

In [14]:
# Plus ce score est grand moins on est content
def error(real_category, predicted_category):
    # On a trouvé la bonne catégorie
    if real_category == predicted_category:
        return 0
    
    # On extrait les sous catégories
    real_categories = real_category.split(" > ")
    pred_categories = predicted_category.split(" > ")
    # On flag les catégories adultes
    adult_categories = ['ADULTE - EROTIQUE', 'VIN - ALCOOL - LIQUIDES']
    real_is_adult = real_categories[0] in adult_categories
    pred_is_adult = pred_categories[0] in adult_categories
    
    # Attention non symmétrie de l'erreur !
    if real_is_adult and not pred_is_adult:
        error = 10_000
        return error
    
    # On identifie à quel niveau on s'est trompé
    for real, pred, error in zip(real_categories, pred_categories, [100, 10, 1]):
        if real != pred:
            return error
    raise ValueError("Catégories différentes, mais aucune différence trouvée")

# Une prédiction moins dummy

Le dataset est bien plus volumineux que les datasets utilisés en TP jusqu'à aujourd'hui.

Si vous vous lancez tête baissée à générer une prédiction vous allez attendre longtemps d'obtenir vos résultats. Il y a 2 raisons à cela : 
- Le nombre de lignes de l'ensemble train est très important
- Le nombre de classes à prédire est aussi très grand. Certains classifier multi-classe génèrent des classifier binaires de type One vs Rest. Si vous avez 600 classes vous allez apprendre 600 classifiers différents

Simplifions donc ces 2 problèmes.

## 1. Travaillons sur 10% du dataset de train

Commencons par itérer rapidement sur le jeu de données. Une fois qu'on aurra un modèle qui nous convient on pourra travailler sur plus de volumétrie.

Générez train_subset, un sample de train faisant 10% de sa taille

In [83]:
train_subset_size = int(0.1*train.shape[0])
chosen_idx = np.random.choice(train_subset_size, replace=False, size=train_subset_size)
train_subset = train.iloc[chosen_idx]

## 2. Limitons les classes

600 classes c'est beaucoup trop. 

On a vu que les catégories contiennent une hiérarchie "catégorie 1 > catégorie 2 > catégorie 3", commencons par travailler uniquement sur les catégories 1.

Générez la colonne "category_1" dans train_subset à partir de la colonne "category" qui contient uniquement la catégorie 1 du produit

In [84]:
def first_category(full_category):
    return full_category.split(">")[0].strip()
train_subset.insert(train_subset.shape[1], "category_1", list(map(first_category, train_subset["category"])))
train_subset

,idp,title,description,price,category,category_1
13478,296964,Haba 303105 Soplar el pastel,Nouveau produit pour les enfants,18.82,JEUX - JOUETS > POUPEE - PELUCHE > VETEMENT - ...,JEUX - JOUETS
22287,167222,Sac Squaremouth Katana Cuir de Vachette 83254 ...,Cet élégant sac squaremouth Katana en cuir de ...,160.00,BAGAGERIE > BAGAGES > SAC DE VOYAGE,BAGAGERIE
21222,219168,Pupa vernis à ongles lasting color Transparent...,Pupa vernis à ongles lasting color Transparent...,18.63,HYGIENE - BEAUTE - PARFUM > ONGLERIE > VERNIS ...,HYGIENE - BEAUTE - PARFUM
29395,28421,80 260V Multimètre de surveillance numérique C...,80★260V multimètre de surveillance AC 100A num...,8.49,BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL ...,BRICOLAGE - OUTILLAGE - QUINCAILLERIE
4964,521046,Solar Power Led Under Ground lumière Jardin co...,"Spécifications :Matériau : plastique, métalTai...",16.59,JARDIN - PISCINE > LUMINAIRE D'EXTERIEUR > LAM...,JARDIN - PISCINE
...,...,...,...,...,...,...
13931,828454,Toolbox : pour la pratique du médecin de famille,"Composé de 22 chapitres denses, TOOLBOX fait s...",56.00,LIBRAIRIE > MEDECINE > ETUDE MEDICALE - CONCOU...,LIBRAIRIE
15699,209274,Full Hd 720P Mini Dv Dvr Caméra mini Caméscope...,Nom de la marque: YOGO Support Haute Définiti...,24.99,PHOTO - OPTIQUE > CAMESCOPE > CAMERA MINIATURE...,PHOTO - OPTIQUE
25907,715464,100 Walks in Surrey,"Surrey is a walker's paradise, with rolling ch...",13.42,JEUX VIDEO > ACCESSOIRES JEUX VIDEO - ACCESSOI...,JEUX VIDEO
5988,220988,6pcs Cordes Ferrules Viroles Montage Bagues Fi...,Ficelles de corps de guitare électrique à tr...,8.01,INSTRUMENTS DE MUSIQUE > INSTRUMENT A CORDES >...,INSTRUMENTS DE MUSIQUE


In [100]:
categories_freq = {}
for el in train_subset.iterrows():
    cat1 = el[1]["category_1"]
    if cat1 not in categories_freq:
        categories_freq[cat1] = {"max_freq" : 0, "max_cat" : el[1]["category"]}
    
    if el[1]["category"] not in categories_freq[cat1]:
        categories_freq[cat1][el[1]["category"]] = 1
    else :
        categories_freq[cat1][el[1]["category"]] += 1
        if categories_freq[cat1]["max_freq"] < categories_freq[cat1][el[1]["category"]]:
            categories_freq[cat1]["max_freq"] += 1 
            categories_freq[cat1]["max_cat"] = el[1]["category"]


Combien de "category_1" distinctes vous avez?

C'est bien plus raisonnable pour commencer.

## 3. Oh oh il y a pleins de features.

Il y a pleins de features différentes.

- 2 champs de texte title et description 
- 1 champ float : price.

On va se contenter de travailler uniquement avec "title" pour commencer.

Générez les features : X_train

In [49]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

def process_text(line):
    tokens = nltk.word_tokenize(line)
    new_tokens = []
    pattern = '\w'
    for token in tokens:
        if len(token) > 1 and token not in stop_words and re.match(pattern, token):
            new_tokens.append(token.lower())
    return new_tokens

train_subset.insert(train_subset.shape[1], "title_tokens", list(map(process_text, train_subset["title"])))
train_subset

[nltk_data] Downloading package stopwords to /home/cisd-
[nltk_data]     calluau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,idp,title,description,price,category,category_1,title_tokens
7088,619540,Embout double d'extrémité de câble Lapp 618019...,NaN,15.99,BRICOLAGE - OUTILLAGE - QUINCAILLERIE > ELECTR...,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,"[embout, double, d'extrémité, câble, lapp, 618..."
15596,64037,The Sermon on Exposition Boulevard [Limited],The Sermon on Exposition Boulevard [Limited] ...,21.93,MUSIQUE > CD > CD POP ROCK - CD ROCK INDE,MUSIQUE,"[the, sermon, exposition, boulevard, limited]"
13010,360271,Robinet de Chauffage Instantané Électrique 360...,Liste de colisage:🌲1 x robinet🌲1 x chauffage🌲1...,43.82,BRICOLAGE - OUTILLAGE - QUINCAILLERIE > SANITA...,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,"[robinet, chauffage, instantané, électrique, 3..."
16015,280937,Fil Pla 3 mm Vert Clair 1 kg,Fil pour imprimantes 3D (K8200).PLA est un pol...,30.24,INFORMATIQUE > IMPRESSION - SCANNER > FIL POUR...,INFORMATIQUE,"[fil, pla, mm, vert, clair, kg]"
29221,6933,Neufu Tapis De Cuisine Antidérapant Moderne 60...,Emballage inclus： 1 x tapis antidérapant Cara...,15.99,DECO - LINGE - LUMINAIRE > DECORATION DU SOL >...,DECO - LINGE - LUMINAIRE,"[neufu, tapis, de, cuisine, antidérapant, mode..."
...,...,...,...,...,...,...,...
13705,895860,"A chacun son tour, chroniques du tour de France","A chacun son tour, chroniques du tour de Franc...",17.00,LIBRAIRIE > LOISIRS - JEUX - SPORT > SPORT,LIBRAIRIE,"[chacun, tour, chroniques, tour, france]"
3364,5301,Énergie solaire danse animal Swinging peluches...,Mélangisme animal solaire Danse Powered Animat...,0.82,AUTO - MOTO > CONFORT CONDUCTEUR ET PASSAGER >...,AUTO - MOTO,"[énergie, solaire, danse, animal, swinging, pe..."
23298,420023,Funmoon Slim Costume 2 Pièces Pour Hommes Robe...,Slim Costume 2 pièces pour hommes Robe Casual ...,52.00,VETEMENTS - LINGERIE > MANTEAU - VESTE > VESTE...,VETEMENTS - LINGERIE,"[funmoon, slim, costume, pièces, pour, hommes,..."
11968,39065,Table Basse avec Design Exclusif à 3 Couches d...,Table Basse avec Design Exclusif à 3 Couches d...,63.99,MEUBLE > MEUBLE DE SEJOUR - ENTREE > TABLE BASSE,MEUBLE,"[table, basse, design, exclusif, couches, salo..."


In [108]:
def assign_id_to_words(data, field):
    words_to_id = {}
    nextid = 0
    for row in data[field]:
        for word in row:
            if word not in words_to_id:
                words_to_id[word] = nextid
                nextid += 1
    return words_to_id

test.insert(test.shape[1], "title_tokens", list(map(process_text, test["title"])))

words_to_id = assign_id_to_words(pd.concat([train, test]), "title_tokens")

categories_to_id = {}
nextid = 0
for cat in train_subset["category_1"]:
    if cat not in categories_to_id:
         categories_to_id[cat] = nextid
         nextid += 1

ValueError: cannot insert title_tokens, already exists

In [103]:
id_to_category = {v: k for k, v in categories_to_id.items()}

In [73]:
def bag_of_words_to_vector(bag_of_words, words_to_id):
    features = np.zeros(len(words_to_id))
    for word in bag_of_words:
        features[words_to_id[word]] = 1
    return features

In [74]:
X_train = []
y_train = []
for row in train_subset.iterrows():
    X_train.append(bag_of_words_to_vector(row[1]["title_tokens"], words_to_id))
    y_train.append(categories_to_id[row[1]["category_1"]])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [75]:
X_train

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [76]:
y_train

array([ 0,  1,  0, ..., 20, 19,  0])

## 4. Calculez un modèle avec ces simplifications

Cf TP1 et TP2 : Entrainez une régression Logistique

In [79]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## 5. Calculez votre prédiction

Appliquez votre modèle sur les données test

In [102]:
confusion_matrix = sklearn.metrics.confusion_matrix(y_train, model.predict(X_train))


In [81]:
sklearn.model_selection.cross_validate(model, X_train, y_train, cv=5)

/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modu

{'fit_time': array([136.85059667, 136.19317913, 152.77317333, 152.77273512,
        137.79722834]),
 'score_time': array([3.14460659, 1.05571818, 2.36640215, 1.05708647, 0.96698594]),
 'test_score': array([0.81666667, 0.81733333, 0.81933333, 0.81433333, 0.8125    ])}

## 6. Soumettez votre prédiction.

*Pas si vite* : Vous ne prédisez que la catégorie 1. Le script d'évaluation attend une catégorie complète...

C'est simple, pour chaque catégorie 1, choisissez la catégorie de votre choix qui commence par cette "categorie 1".

Modifiez votre prédiction, y_pred, en conséquence.

Soumettez votre prédiction :

In [105]:
test

,idp,title,description,price
0,4873686,Endoscope Sans Fil Wifi 1200 P Caméra D'inspec...,"Descriptions&nbsp;Résolution: 640 * 480, 1280 ...",16.85
1,6320508,Aider l'enfant dyslexique,Bernard Jumel 3e édition,16.90
2,6351042,Modà ̈Le réduit de véhicule de construction Li...,Modà¨le réduit de véhicule de construction Lie...,24.97
3,3853418,Lampenwelt lampadaire Led à intensité variable...,Ce lampadaire séduit par son design esthétique...,167.90
4,6373582,"Caméra vidéo caméscope, gordvec 2,7 Pouces écr...","""Caractéristiques : 2,7 pouces (16 : 9) LCD, l...",98.99
...,...,...,...,...
999995,6036736,Grille Pomme De Douche (5) Presto 90164,GRILLE POMME DE DOUCHE (5) PRESTO 90164 Ce s...,7.15
999996,761492,Brosse pinceau pour correcteurs et anti cernes,Brosse professionnelle de maquillage eetagrave...,9.90
999997,5165063,La ruine et le geste architectural,La ruine et le geste architectural Pierre Hy...,20.00
999998,3292782,Cen Robinet G1 2 robinet de bassin de lavabo b...,"☻☻Facile à installer, nécessite seulement une ...",31.84


In [106]:
X_test = []

for row in test.iterrows():
    X_test.append(bag_of_words_to_vector(row[1]["title_tokens"], words_to_id))

X_test = np.array(X_test)

KeyError: 'dyslexique'

In [98]:
X_test

In [98]:
y_temp = model.predict(X_test)

categories_freq
y_pred = list(map(lambda cat_id: categories_freq[id_to_category[cat_id]]["max_cat"], y_temp))

prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})

['BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',
 'MUSIQUE > CD > CD VARIETE INTERNATIONALE',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',
 'INFORMATIQUE > CONNECTIQUE - ALIMENTATION > CABLE AUDIO VIDEO',
 'DECO - LINGE - LUMINAIRE > ARTICLES - DECORATION DE FETE > BALLON DECORATIF - POMPE POUR BALLON',
 'HYGIENE - BEAUTE - PARFUM > CAPILLAIRE > BANDEAU - SERRE-TETE - HEADBAND - HAIRBAND',
 'JEUX - JOUETS > VEHICULE POUR ENFANT > ACCESSOIRE - PIECE DETACHEE VEHICULE',
 'TV - VIDEO - SON > LECTEUR MUSIQUE > LECTEUR MP3',
 'BAGAGERIE > MAROQUINERIE > POCHETTE A MAIN',
 'INSTRUMENTS DE MUSIQUE > MATERIEL DE JEU > PARTITION',
 'INFORMATIQUE > CONNECTIQUE - ALIMENTATION > CABLE AUDIO VIDEO',
 'CHAUSSURES - ACCESSOIRES > BASKET - SPORTSWEAR > BASKET',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',
 'ANIMALERIE > JOUET > JOUET',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',

Vous pouvez regardez vos scores en exécutant le notebook [Leaderboard.ipynb](Leaderboard.ipynb). Les données sont mises à jour toutes les 30min.

# Sauvegardez votre modèle

Pour ne pas devoir recommencer à chaque fois tout ce dur labeur, et cette longue attente, vous pouvez sauvegarder votre modèle et votre vectorizer.

La prochaine fois vous n'aurez qu'à les recharger pour faire directement vos prédictions (c'est ce qui est attendu pour la soutenance, sinon le timing sera trop serré) 

La documentation : https://scikit-learn.org/stable/modules/model_persistence.html

# Maintenant à vous de jouer

Vous pouvez commencer par travailler sur plus de volumétrie que 5% du dataset. Mais maintenant le challenge commence.

Si vous ne savez pas par où commencer suivez le déroulement des 2 premiers TPs, en prenant garde à la volumétrie. Ils sont disponibles dans le répertoire ~cisd-jacq/TP/

Contrairement aux TPs vous n'avez pas d'information sur les données de test. (Mis à part le score calculé toute les 30min).

Pour évaluer votre modèle et l'améliorer vous pouvez utiliser les données de train pour crééer un ensemble d'entrainement et un ensemble de validation. 

Vous pourrez alors évaluer plus rapidement vos modèles et identifier quelles sont les catégories sur lesquelles vous devez vous améliorer.

Il n'y a pas que la Régression Logistique dans la vie, essayez d'autres modèles. Je vous ai fait travailler avec sklearn, mais il existe aussi d'autres librairies.

Pour paralléliser vos calculs :
- multiprocessing : https://docs.python.org/3/library/multiprocessing.html
- dask : https://dask.org/ + https://distributed.dask.org/en/latest/ 